In [1]:
!pip install sklearn

You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [2]:
import numpy as np
import datetime
import tensorflow as tf
from tensorflow.keras.datasets import fashion_mnist

from sklearn.preprocessing import LabelBinarizer

# Coleta dos Dados

In [3]:
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

In [4]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((60000, 28, 28), (60000,), (10000, 28, 28), (10000,))

## Normalização das imagens

In [5]:
X_train = X_train / 255.0
X_test = X_test / 255.0

## Remodelagem (reshaping) a base de dados

In [6]:
# 6000 imagens, 28 de altura e 28 de largura
X_train.shape

(60000, 28, 28)

In [7]:
# altera de uma matriz para um vetor
X_train = X_train.reshape(-1, 28*28)
X_test = X_test.reshape(-1, 28*28)
X_train.shape, X_test.shape

((60000, 784), (10000, 784))

In [8]:
lb = LabelBinarizer()
y_train = lb.fit_transform(y_train)
y_test = lb.fit_transform(y_test)

In [9]:
lr = 0.01 
comms_round = 100
loss='categorical_crossentropy'
metrics = ['accuracy']
optimizer = tf.keras.optimizers.SGD(lr=lr, decay= lr/comms_round, momentum=0.9)    

# Criação de Clientes

In [10]:
n_clients = 20

1. Cria um vetor com os indices dos pontos de dados e embaralha seus indices
2. Distribui os indices dos pontos de dados entre os clientes e então embaralha-os
3. Gera valores a partir de uma distribuição uniforme (entre 0 e 1)
4. Passa por metade dos indices de clientes, removendo uma pocentagem dos dados de cada um desses clientes e passando para a outra metade dos clientes 

In [11]:
def generate_indexes(X, n_clients):
    indexes = np.arange(np.random.randint(int(X.shape[0] * 0.8), X.shape[0]))
    
    np.random.shuffle(indexes)
    indexes = np.array_split(indexes, n_clients)
    
    clients_index = np.arange(n_clients)
    np.random.shuffle(clients_index)
    
    half_clients = int(n_clients / 2)
    samples = np.random.random_sample((half_clients,))
    for i in range(half_clients):
        client_A = indexes[clients_index[i]]
        client_B = indexes[clients_index[i + half_clients]]
        
        client_A = np.concatenate((client_A, client_B[:int(client_B.shape[0] * samples[i])]))
        client_B = client_B[int(client_B.shape[0] * samples[i]):]
        
        indexes[clients_index[i]] = client_A
        indexes[clients_index[i + half_clients]] = client_B
    
    return indexes

In [12]:
def create_clients(X, y, n_clients):

    indexes = generate_indexes(X, n_clients)
    
    X_slices = np.array([X[indexes[i]] for i in range(n_clients)], dtype='object')
    y_slices = np.array([y[indexes[i]] for i in range(n_clients)], dtype='object')
    return X_slices, y_slices

In [13]:
X_slices, y_slices = create_clients(X_train, y_train, n_clients)

In [14]:
X_slices.shape, y_slices.shape

((20,), (20,))

# Construção do Modelo

In [15]:
class MLP:
    def build(self):
        model = tf.keras.models.Sequential()
        model.add(tf.keras.layers.Dense(units=256, activation='relu', input_shape=(784, )))
        model.add(tf.keras.layers.Dropout(0.2))
        model.add(tf.keras.layers.Dense(units=128, activation='relu', input_shape=(784, )))
        model.add(tf.keras.layers.Dropout(0.2))
        model.add(tf.keras.layers.Dense(units=10, activation='softmax'))
        return model

In [16]:
def NN_update(local_data, global_data, model, X_test, y_test):
    
    weights = (local_data / global_data) * np.array(client_model.get_weights(), dtype='object')
    test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
    
    test_loss = (local_data / global_data) * test_loss
    test_accuracy = (local_data / global_data) * test_accuracy
    
    return weights, test_loss, test_accuracy

In [17]:
mlp = MLP()

global_model = mlp.build()
global_model.compile(loss=loss, optimizer=optimizer, metrics=metrics)

for j in range(comms_round):

    global_weights = global_model.get_weights()
    new_global_weights = [np.zeros(weight.shape) for weight in global_weights]
    
    X_slices, y_slices = create_clients(X_train, y_train, n_clients)
    global_data = sum([size.shape[0] for size in X_slices])
    
    avg_local_loss = 0.0
    avg_local_acc = 0.0
    
    for i in range(n_clients):
        client_model = mlp.build()
        client_model.compile(loss=loss, 
                      optimizer=optimizer, 
                      metrics=metrics)
        
        client_model.set_weights(global_weights)
        client_model.fit(X_slices[i], y_slices[i], epochs=2, verbose=0)
        
        weights, test_loss, test_accuracy = NN_update(X_slices[i].shape[0], global_data, 
                                                      client_model, X_test, y_test)
        
        new_global_weights += np.array(weights, dtype='object')
        avg_local_loss += test_loss
        avg_local_acc += test_accuracy
    
    global_model.set_weights(new_global_weights) #
    
    test_loss, test_accuracy = global_model.evaluate(X_test, y_test)
    print("Round {}, Loss: {:.3f}, Accuracy: {:.3f}".format(j, test_loss, test_accuracy))
    print("(AVG Local) Round {}, Loss: {:.3f}, Accuracy: {:.3f}".format(j, avg_local_loss, avg_local_acc))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:29: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


313/313 [==============================] - 0s 1ms/step - loss: 0.5989 - accuracy: 0.7869
Round 0, Loss: 0.602, Accuracy: 0.787
(AVG Local) Round 0, Loss: 0.652, Accuracy: 0.762
313/313 [==============================] - 0s 1ms/step - loss: 0.5309 - accuracy: 0.8087
Round 1, Loss: 0.531, Accuracy: 0.809
(AVG Local) Round 1, Loss: 0.578, Accuracy: 0.788
313/313 [==============================] - 0s 1ms/step - loss: 0.4942 - accuracy: 0.8235
Round 2, Loss: 0.494, Accuracy: 0.823
(AVG Local) Round 2, Loss: 0.524, Accuracy: 0.811
313/313 [==============================] - 0s 1ms/step - loss: 0.4734 - accuracy: 0.8301
Round 3, Loss: 0.473, Accuracy: 0.830
(AVG Local) Round 3, Loss: 0.501, Accuracy: 0.820
313/313 [==============================] - 0s 1ms/step - loss: 0.4578 - accuracy: 0.8370
Round 4, Loss: 0.458, Accuracy: 0.837
(AVG Local) Round 4, Loss: 0.480, Accuracy: 0.827
313/313 [==============================] - 0s 1ms/step - loss: 0.4430 - accuracy: 0.8422
Round 5, Loss: 0.443, Accu

313/313 [==============================] - 0s 1ms/step - loss: 0.3736 - accuracy: 0.8649
Round 46, Loss: 0.374, Accuracy: 0.865
(AVG Local) Round 46, Loss: 0.378, Accuracy: 0.863
313/313 [==============================] - 0s 1ms/step - loss: 0.3726 - accuracy: 0.8658
Round 47, Loss: 0.373, Accuracy: 0.866
(AVG Local) Round 47, Loss: 0.376, Accuracy: 0.864
313/313 [==============================] - 0s 1ms/step - loss: 0.3726 - accuracy: 0.8656
Round 48, Loss: 0.373, Accuracy: 0.866
(AVG Local) Round 48, Loss: 0.377, Accuracy: 0.864
313/313 [==============================] - 0s 1ms/step - loss: 0.3721 - accuracy: 0.8660
Round 49, Loss: 0.372, Accuracy: 0.866
(AVG Local) Round 49, Loss: 0.376, Accuracy: 0.864
313/313 [==============================] - 0s 1ms/step - loss: 0.3715 - accuracy: 0.8660
Round 50, Loss: 0.371, Accuracy: 0.866
(AVG Local) Round 50, Loss: 0.376, Accuracy: 0.864
313/313 [==============================] - 0s 1ms/step - loss: 0.3709 - accuracy: 0.8665
Round 51, Loss: 

KeyboardInterrupt: 